In [1]:
import pickle
import gensim
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from collections import defaultdict

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
subm = pd.read_csv('data/sample_submission.csv')

In [5]:
l1 = [1,2,3]
l2 = [4,5,6]
l3 = [7,8,9]

for a1,a2,a3 in product(l1,l2,l3):
    print(a1,a2,a3)

1 4 7
1 4 8
1 4 9
1 5 7
1 5 8
1 5 9
1 6 7
1 6 8
1 6 9
2 4 7
2 4 8
2 4 9
2 5 7
2 5 8
2 5 9
2 6 7
2 6 8
2 6 9
3 4 7
3 4 8
3 4 9
3 5 7
3 5 8
3 5 9
3 6 7
3 6 8
3 6 9


In [5]:
'''
Arguments:
sentences=None, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, 
sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5,
cbow_mean=1, hashfxn=<built-in function hash>, iter=5, null_word=0, trim_rule=None, 
sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=()
'''
sg=1
size=300
window=10
min_count=3
# model = gensim.models.word2vec.Word2Vec(array_to_list(train_tokens), 
#                                         sg=sg, size=size, window=window, min_count=min_count)

In [10]:
model_name = '{0}-sz{1}-win{2}-minc{3}'.format('sg' if sg==1 else 'cbow', size, window, min_count)
#model.save('data/w2v-' + model_name + '.model')

In [4]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.vectors[0])

    def fit(self, X, y):
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [6]:
model = gensim.models.word2vec.Word2Vec.load('data/w2v-sg-sz300-win10-minc3.model')

In [7]:
embedding_vectorizer = TfidfEmbeddingVectorizer(model.wv)
train_embedded = embedding_vectorizer.fit(train_tokens, None)

In [8]:
train_embedded = embedding_vectorizer.transform(train_tokens)
test_embedded = embedding_vectorizer.transform(test_tokens)

In [9]:
from sklearn.linear_model import LogisticRegression

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((len(test), len(label_cols)))
for i, j in enumerate(label_cols):
    m = LogisticRegression()
    mf = m.fit(train_embedded, train[j])
    preds[:,i] = mf.predict_proba(test_embedded)[:,1]

In [11]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission/submission-toxicw2v-doctfidf-lr-{}.csv'.format(model_name), index=False)